# The Analyzer

In [11]:
# Libraries
import folium
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Adding geolocations for all production countries / regions

### Preparing data from csv to plot to map

In [12]:
# Read the data to a pandas data frame
df1 = pd.read_csv('wine_data.csv', 
                 sep=',', 
                 encoding='utf-8')[['land', 'lat', 'lon']]
# Preparing data, so it doesn't plot duplicate values more than once
df2 = df1.drop_duplicates()
df3 = df2.reset_index(drop=True)
df3.head()

,land,lat,lon
0,USA,39.783730,-100.445882
1,Italien,42.638426,12.674297
2,Argentinien,-34.996496,-64.967282
3,Frankreich,46.603354,1.888334
4,Chile,-31.761336,-71.318770


### Plotting the map

In [13]:
# Initalises the map
m = folium.Map(location=[47.44, 8.65], zoom_start=10)

# Add lat/lon of addresses
for i in range(0, len(df3)):
    folium.Marker(location=(df3.iloc[i]['lat'], 
                            df3.iloc[i]['lon']), 
                  popup=df3.iloc[i]['land']).add_to(m)

# Layer control
folium.LayerControl().add_to(m)

# Plot map
m